**ResnetNet**

In [ ]:
import  tensorflow as tf
from    tensorflow import keras
from    tensorflow.keras import layers, Sequential
from tqdm import tqdm_notebook

import  tensorflow as tf
from    tensorflow.keras import layers, optimizers, datasets, Sequential
import  os

os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
tf.random.set_seed(2345)

def preprocess(x, y):
    # 将数据映射到-1~1
    x = 2*tf.cast(x, dtype=tf.float32) / 255. - 1
    y = tf.cast(y, dtype=tf.int32) # 类型转换
    return x,y

class BasicBlock(layers.Layer):
    # 残差模块
    def __init__(self, filter_num, stride=1):
        super(BasicBlock, self).__init__()
        # 第一个卷积单元
        self.conv1 = layers.Conv2D(filter_num, (3, 3), strides=stride, padding='same')
        self.bn1 = layers.BatchNormalization()
        self.relu = layers.Activation('relu')
        # 第二个卷积单元
        self.conv2 = layers.Conv2D(filter_num, (3, 3), strides=1, padding='same')
        self.bn2 = layers.BatchNormalization()

        if stride != 1:# 通过1x1卷积完成shape匹配
            self.downsample = Sequential()
            self.downsample.add(layers.Conv2D(filter_num, (1, 1), strides=stride))
        else:# shape匹配，直接短接
            self.downsample = lambda x:x

    def call(self, inputs, training=None):

        # [b, h, w, c]，通过第一个卷积单元
        out = self.conv1(inputs)
        out = self.bn1(out)
        out = self.relu(out)
        # 通过第二个卷积单元
        out = self.conv2(out)
        out = self.bn2(out)
        # 通过identity模块
        identity = self.downsample(inputs)
        # 2条路径输出直接相加
        output = layers.add([out, identity])
        output = tf.nn.relu(output) # 激活函数

        return output

class ResNet(keras.Model):
    # 通用的ResNet实现类
    def __init__(self, layer_dims, num_classes=10): # [2, 2, 2, 2]
        super(ResNet, self).__init__()
        # 根网络，预处理
        self.stem = Sequential([layers.Conv2D(64, (3, 3), strides=(1, 1)),
                    layers.BatchNormalization(),
                    layers.Activation('relu'),
                    layers.MaxPool2D(pool_size=(2, 2), strides=(1, 1), padding='same')
                    ])
        # 堆叠4个Block，每个block包含了多个BasicBlock,设置步长不一样
        self.layer1 = self.build_resblock(64,  layer_dims[0])
        self.layer2 = self.build_resblock(128, layer_dims[1], stride=2)
        self.layer3 = self.build_resblock(256, layer_dims[2], stride=2)
        self.layer4 = self.build_resblock(512, layer_dims[3], stride=2)

        # 通过Pooling层将高宽降低为1x1
        self.avgpool = layers.GlobalAveragePooling2D()
        # 最后连接一个全连接层分类
        self.fc = layers.Dense(num_classes)

    def call(self, inputs, training=None):
        # 通过根网络
        x = self.stem(inputs)
        # 一次通过4个模块
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        # 通过池化层
        x = self.avgpool(x)
        # 通过全连接层
        x = self.fc(x)

        return x

    def build_resblock(self, filter_num, blocks, stride=1):
        # 辅助函数，堆叠filter_num个BasicBlock
        res_blocks = Sequential()
        # 只有第一个BasicBlock的步长可能不为1，实现下采样
        res_blocks.add(BasicBlock(filter_num, stride))

        for _ in range(1, blocks):#其他BasicBlock步长都为1
            res_blocks.add(BasicBlock(filter_num, stride=1))

        return res_blocks

In [ ]:
(x,y), (x_test, y_test) = datasets.cifar10.load_data() # 加载数据集
y = tf.squeeze(y, axis=1) # 删除不必要的维度
y_test = tf.squeeze(y_test, axis=1) # 删除不必要的维度
print(x.shape, y.shape, x_test.shape, y_test.shape)

train_db = tf.data.Dataset.from_tensor_slices((x,y)) # 构建训练集
# 随机打散，预处理，批量化
train_db = train_db.shuffle(1000).map(preprocess).batch(512)

test_db = tf.data.Dataset.from_tensor_slices((x_test,y_test)) #构建测试集
# 随机打散，预处理，批量化
test_db = test_db.map(preprocess).batch(512)
# 采样一个样本
sample = next(iter(train_db))
print('sample:', sample[0].shape, sample[1].shape,
      tf.reduce_min(sample[0]), tf.reduce_max(sample[0]))

170508288/170498071 [==============================] - 6s 0us/step
(50000, 32, 32, 3) (50000,) (10000, 32, 32, 3) (10000,)
sample: (512, 32, 32, 3) (512,) tf.Tensor(-1.0, shape=(), dtype=float32) tf.Tensor(1.0, shape=(), dtype=float32)


In [ ]:
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')

# [b, 32, 32, 3] => [b, 1, 1, 512]
model = ResNet([2, 2, 2, 2]) # ResNet18网络
# model = ResNet([3, 4, 6, 3]) # ResNet34网络
model.build(input_shape=(None, 32, 32, 3))
model.summary() # 统计网络参数
optimizer = optimizers.Adam(lr=1e-4) # 构建优化器

for epoch in tqdm_notebook(range(100)): # 训练epoch
    for step, (x,y) in enumerate(train_db):
        with tf.GradientTape() as tape:
            # [b, 32, 32, 3] => [b, 10],前向传播
            logits = model(x)
            train_accuracy(y,logits)
            # [b] => [b, 10],one-hot编码
            y_onehot = tf.one_hot(y, depth=10)
            # 计算交叉熵
            loss = tf.losses.categorical_crossentropy(y_onehot, logits, from_logits=True)
            loss = tf.reduce_mean(loss)
        # 计算梯度信息
        grads = tape.gradient(loss, model.trainable_variables)
        # 更新网络参数
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if step %50 == 0:
            print(epoch, step, 'train loss:', float(loss),'train acc: {:.4f}'.format(train_accuracy.result()*100))
            train_accuracy.reset_states()


    total_num = 0
    total_correct = 0
    for x,y in test_db:

        logits = model(x)
        loss = tf.losses.sparse_categorical_crossentropy(y, logits, from_logits=True)
        test_loss(loss)
        
        prob = tf.nn.softmax(logits, axis=1)
        pred = tf.argmax(prob, axis=1)
        pred = tf.cast(pred, dtype=tf.int32)
        
        correct = tf.cast(tf.equal(pred, y), dtype=tf.int32)
        correct = tf.reduce_sum(correct)

        total_num += x.shape[0]
        total_correct += int(correct)

    acc = total_correct / total_num
    print(epoch,'test loss:{:.4f}'.format(test_loss.result()), 'test acc:', acc)
    test_loss.reset_states()

Model: "res_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 30, 30, 64)        2048      
_________________________________________________________________
sequential_1 (Sequential)    (None, 30, 30, 64)        148736    
_________________________________________________________________
sequential_2 (Sequential)    (None, 15, 15, 128)       526976    
_________________________________________________________________
sequential_4 (Sequential)    (None, 8, 8, 256)         2102528   
_________________________________________________________________
sequential_6 (Sequential)    (None, 4, 4, 512)         8399360   
_________________________________________________________________
global_average_pooling2d (Gl multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  5130

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


  0%|          | 0/100 [00:00<?, ?it/s]

0 0 train loss: 2.304671287536621 train acc: 12.1094
0 50 train loss: 1.7234169244766235 train acc: 27.0273
0 test loss:1.5824 test acc: 0.41
1 0 train loss: 1.5249770879745483 train acc: 37.5533
1 50 train loss: 1.4251024723052979 train acc: 42.7812
1 test loss:1.4177 test acc: 0.48
2 0 train loss: 1.413888931274414 train acc: 45.9918
2 50 train loss: 1.3886022567749023 train acc: 49.5352
2 test loss:1.2942 test acc: 0.532
3 0 train loss: 1.1906986236572266 train acc: 52.0697
3 50 train loss: 1.1259057521820068 train acc: 54.3398
3 test loss:1.1543 test acc: 0.5824
4 0 train loss: 1.1469507217407227 train acc: 56.5861
4 50 train loss: 1.0890780687332153 train acc: 59.0977
4 test loss:1.0486 test acc: 0.6226
5 0 train loss: 0.9929785132408142 train acc: 61.1189
5 50 train loss: 1.0194835662841797 train acc: 62.1680
5 test loss:1.0009 test acc: 0.6398
6 0 train loss: 0.990096390247345 train acc: 63.0656
6 50 train loss: 1.0571194887161255 train acc: 64.9102
6 test loss:1.0650 test acc: 